In [1]:
import langchain
from langchain_community.document_loaders import DataFrameLoader
import json
import pandas as pd
import getpass
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings 
import openai 
from langchain_core.documents import Document

c:\Users\lucas\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
with open('subject_areas.json', 'r') as f:
    subject_areas = json.load(f)

In [8]:
subject_dict = [{k: v for k, v in d.items() if k in ['code', 'title'] } for d in subject_areas]

In [9]:
print(subject_dict[1])
print(len(subject_dict))

{'code': 'A E', 'title': 'AEROSPACE ENGINEERING'}
469


In [10]:
abbrv_desc = [f'{title}: {code}' for code, title in zip([d['code'] for d in subject_dict], [d['title'] for d in subject_dict])]

AERONAUTICS & ASTRONAUTICS: A A


In [16]:
departments = ['SWA', 'ARCTIC', 'TAGLG', 'AAS', 'CHSTU', 'BENG', 'CFRM', 'AFRAM', 'AES', 'ASIAN',
 'AMATH', 'ART', 'DESIGN', 'AIS', 'INDO', 'INDN', 'ARCHY', 'URDU', 'HINDI', 'SNKRT',
 'CHIN', 'ASTBIO', 'KOREAN', 'HUM', 'VIET', 'JAPAN', 'ASTR', 'CMS', 'GREEK', 'CLAS',
 'LATIN', 'DXARTS', 'ANTH', 'INDIV', 'CHID', 'CHEM', 'TXTDS', 'BIOL', 'DANCE',
 'COM', 'FRENCH', 'ITAL', 'HSTAFM', 'DRAMA', 'CSE', 'ECON', 'HSTLAC', 'HSTAM',
 'HPS', 'HSTRY', 'GWSS', 'HSTAS', 'HSTCMP', 'LABOR', 'GEOG', 'ASL', 'HSTEU',
 'GERMAN', 'RELIG', 'ARAMIC', 'COPTIC', 'GEEZ', 'EGYPT', 'LADINO', 'MICROM', 'LSJ',
 'JSIS', 'ARAB', 'TURKIC', 'CHGTAI', 'LING', 'BIBHEB', 'MODHEB', 'PRSAN', 'MATH',
 'ENGL', 'KYRGYZ', 'KAZAKH', 'UGARIT', 'UZBEK', 'UYGUR', 'TKISH', 'MUSAP', 'MUSED',
 'NEUSCI', 'ETHICS', 'MUSEN', 'MUHST', 'MELC', 'FINN', 'DANISH', 'ESTO', 'LITH',
 'LATV', 'NORW', 'BCMS', 'SWED', 'CZECH', 'BULGR', 'MUSICP', 'ROMN', 'PHIL',
 'POLSH', 'GLITS', 'UKR', 'SLVN', 'PORT', 'SCAND', 'PHYS', 'SPLING', 'MUSIC',
 'SLAVIC', 'RUSS', 'PSYCH', 'CEP', 'CM', 'SPHSC', 'STAT', 'ACCTG', 'ARCH', 'ENTRE',
 'SOC', 'FIN', 'SPAN', 'QMETH', 'OPMGT', 'MGMT', 'DPHS', 'MKTG', 'ORALM', 'OHS',
 'ORTHO', 'ECE', 'PERIO', 'ECFS', 'EDUC', 'NME', 'EDSPE', 'CEE', 'EDPSY', 'EDLPS',
 'ENGR', 'HCDE', 'BSE', 'FISH', 'FHL', 'MARBIO', 'ENVIR', 'SMEA', 'GRDSCH', 'ESRM',
 'INFO', 'OCEAN', 'ESS', 'LAW', 'NUTR', 'UCONJ', 'BIOC', 'ANEST', 'BIME', 'BIOEN',
 'IMMUN', 'GENOME', 'CONJ', 'PATH', 'MED', 'FAMED', 'OPHTH', 'OTOHN', 'ORTHP',
 'PHCOL', 'IECMH', 'RADGY', 'UROL', 'PBSCI', 'NEURL', 'SURG', 'NMETH', 'NCLIN',
 'PEDS', 'REHAB', 'MEDCH', 'PCEUT', 'PHARM', 'NURS', 'PUBPOL', 'HIHIM', 'BIOST',
 'EPI', 'PABIO', 'PHG', 'SPH', 'LEAD', 'HONORS', 'HSERV']


print(len(departments))



195


In [12]:
#Setting up keys
pinecone_api_key = os.environ.get("pinecone_API")

pc = Pinecone(api_key=pinecone_api_key)

api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key


In [10]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

pc.create_index(
    name= 'department-abbrev-db',
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)



PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '61ad522ff6857dfd325f71e34b875996', 'Date': 'Sat, 22 Feb 2025 22:46:59 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [18]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [21]:
index = pc.Index("department-abbrev-db")
vector_store = PineconeVectorStore(index , embedding = embeddings)

In [22]:
vector_store.delete(delete_all = True)

In [6]:
index = pc.Index("department-abbrev-db")

In [23]:
documents = [Document(page_content = doc) for doc in abbrv_desc]
print(documents[0])

page_content='AERONAUTICS & ASTRONAUTICS: A A'


In [24]:

vector_store.add_documents(documents)



['6cfa42dd-ce46-4abd-8396-400a8d35fe7f',
 'af5d59f5-8a6c-4a33-8ae5-f623c0301404',
 'b0eaceab-9023-4dc9-8b9a-cdc4d31460fd',
 '6ba241b7-81cd-4e3a-9fe0-669c3d333ec1',
 'c9b016e5-b925-44c7-811b-9ea0d897a34a',
 'fa2487ab-8d20-44ce-904b-fd34688ae6ea',
 '38d01b74-493e-45a0-8302-31b0be7cce09',
 'c486f5af-c591-4b34-a768-4d63cbe0f7f4',
 'f98b6199-4c53-480a-88cf-b9a3d21767c4',
 '04885a3a-4050-4b95-82c3-8f7045500489',
 '599d0eb5-43a6-45b5-a0d5-af14209acb2d',
 'addc8994-b068-41a9-972d-67f3e88ec26a',
 '4840bb9b-5210-4d2b-a822-d003ebb4b71f',
 '0212bbcf-aa5b-4a67-becf-781c9071eb25',
 'af48cfb0-511e-4b03-a4ec-a18f2e5720af',
 '821e7cae-fe81-4668-b05f-85247c97925a',
 '802d4fa8-21b8-43bc-9cec-059a990cb2c6',
 '38d7fee8-8cec-4d85-9dd2-0316b37e6aca',
 '8654a0c8-9e1a-4623-8e96-702a45e27d46',
 '0efabc0e-7f63-425d-8b42-70f096cf8af7',
 'b37f37cb-971e-4038-bf8b-62c7c4b41e63',
 '57520aa2-fd3d-4939-ba65-4b65c56cf114',
 '0987de9e-cde8-41f3-b476-d13a8fd31822',
 '231c8c8c-d251-49cd-bdcc-8aa74e671e56',
 '1a1d0ce8-634c-

In [27]:
results = vector_store.similarity_search(
    "What are some urban design courses?",
    k=4,
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* URBAN PLANNING: URBDP [{}]
* URBAN STUDIES - TACOMA: T URB [{}]
* URBAN: URBAN [{}]
* URBAN DESIGN - TACOMA: T UDE [{}]
